In [ ]:
import torch
import numpy as np
from pathlib import Path
from IPython.display import Audio
from loguru import logger

# Fish Speechコンポーネントのインポート
from fish_speech.inference_engine import TTSInferenceEngine
from fish_speech.models.text2semantic.inference import launch_thread_safe_queue
from fish_speech.models.vqgan.inference import load_model as load_decoder_model
from fish_speech.utils.schema import ServeTTSRequest, ServeReferenceAudio

class FishSpeechTTS:
    def __init__(
        self,
        llama_checkpoint_path="checkpoints/fish-speech-1.5",
        decoder_checkpoint_path="checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth",
        decoder_config_name="firefly_gan_vq",
        device="cuda",
        half=False,
        compile=False,
    ):
        """
        Fish Speech TTSエンジンを初期化します。
        
        引数:
            llama_checkpoint_path (str): LLaMAモデルのチェックポイントパス
            decoder_checkpoint_path (str): デコーダーモデルのチェックポイントパス
            decoder_config_name (str): デコーダー設定の名前
            device (str): モデルを実行するデバイス ('cuda'または'cpu')
            half (bool): 半精度（FP16）を使用するかどうか
            compile (bool): モデルをコンパイルするかどうか
        """
        self.device = device
        self.precision = torch.half if half else torch.bfloat16
        self.compile = compile
        
        # MPSまたはCUDAが利用可能かチェックし、デバイスを調整
        if torch.backends.mps.is_available():
            self.device = "mps"
            logger.info("MPSが利用可能です。MPSで実行します。")
        elif not torch.cuda.is_available() and device == "cuda":
            logger.info("CUDAが利用できません。CPUで実行します。")
            self.device = "cpu"
            
        logger.info("LLaMAモデルを読み込んでいます...")
        self.llama_queue = launch_thread_safe_queue(
            checkpoint_path=llama_checkpoint_path,
            device=self.device,
            precision=self.precision,
            compile=self.compile,
        )
        
        logger.info("VQ-GANモデルを読み込んでいます...")
        self.decoder_model = load_decoder_model(
            config_name=decoder_config_name,
            checkpoint_path=decoder_checkpoint_path,
            device=self.device,
        )
        
        logger.info("推論エンジンを作成しています...")
        self.inference_engine = TTSInferenceEngine(
            llama_queue=self.llama_queue,
            decoder_model=self.decoder_model,
            precision=self.precision,
            compile=self.compile,
        )
        
        # ウォームアップ実行
        logger.info("ウォームアップ中...")
        list(
            self.inference_engine.inference(
                ServeTTSRequest(
                    text="Hello world.",
                    references=[],
                    reference_id=None,
                    max_new_tokens=1024,
                    chunk_length=200,
                    top_p=0.7,
                    repetition_penalty=1.5,
                    temperature=0.7,
                    format="wav",
                )
            )
        )
        logger.info("TTSエンジンの準備ができました！")
    
    def text_to_speech(
        self,
        text,
        reference_id=None,
        reference_audio=None,
        reference_text="",
        max_new_tokens=1024,
        chunk_length=200,
        top_p=0.7,
        repetition_penalty=1.2,
        temperature=0.7,
        seed=None,
        use_memory_cache="on",
        return_numpy=True,
    ):
        """
        テキストを音声に変換します。
        
        引数:
            text (str): 音声に変換するテキスト
            reference_id (str, optional): 声のクローニングのための参照ID
            reference_audio (str or bytes, optional): 参照音声ファイルのパスまたは音声バイト
            reference_text (str, optional): 参照音声のためのテキスト
            max_new_tokens (int): 生成する最大トークン数
            chunk_length (int): 合成のためのチャンク長
            top_p (float): Top-pサンプリングパラメータ
            repetition_penalty (float): 繰り返しペナルティ
            temperature (float): サンプリングの温度
            seed (int, optional): 再現性のためのシード
            use_memory_cache (str): メモリキャッシュを使用するかどうか（'on'または'off'）
            return_numpy (bool): numpy配列またはAudioオブジェクトを返すかどうか
            
        戻り値:
            Audio または tuple: IPython.display.Audioオブジェクトまたはタプル（サンプルレート, 音声配列）
        """
        references = []
        if reference_audio:
            if isinstance(reference_audio, str):
                # ファイルパスと仮定
                with open(reference_audio, "rb") as f:
                    audio_bytes = f.read()
            else:
                # すでにバイトデータと仮定
                audio_bytes = reference_audio
                
            references = [ServeReferenceAudio(audio=audio_bytes, text=reference_text)]
        
        req = ServeTTSRequest(
            text=text,
            reference_id=reference_id,
            references=references,
            max_new_tokens=max_new_tokens,
            chunk_length=chunk_length,
            top_p=top_p,
            repetition_penalty=repetition_penalty,
            temperature=temperature,
            seed=seed,
            use_memory_cache=use_memory_cache,
            format="wav",
        )
        
        for result in self.inference_engine.inference(req):
            if result.code == "final":
                sample_rate, audio_data = result.audio
                if return_numpy:
                    return sample_rate, audio_data
                else:
                    return Audio(audio_data, rate=sample_rate)
            elif result.code == "error":
                raise RuntimeError(f"TTSでエラーが発生しました: {result.error}")
        
        raise RuntimeError("音声が生成されませんでした")

# ノートブックでの使用例
# テストするには以下の行をコメント解除してください

# tts = FishSpeechTTS(
#     llama_checkpoint_path="checkpoints/fish-speech-1.5",
#     decoder_checkpoint_path="checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"
# )

# # 簡単なTTSの例
# sample_rate, audio = tts.text_to_speech("こんにちは、これはFish Speechテキスト読み上げのテストです。")
# Audio(audio, rate=sample_rate)

# # 参照音声を使った声のクローニング
# # sample_rate, audio = tts.text_to_speech(
# #     "こんにちは、私はクローンされた声で新しいことを言っています。",
# #     reference_audio="path/to/reference.wav",
# #     reference_text="これは声のクローニングのための参照音声です。"
# # )
# # Audio(audio, rate=sample_rate)

# # 生成された音声をファイルに保存
# # import soundfile as sf
# # sf.write("output.wav", audio, sample_rate)